In [104]:
runtime_meta()

Updated 2018-02-27 14:50:36.395425
By leonyin
Using Python 3.6.1
On Darwin-17.4.0-x86_64-i386-64bit


# USNPL
A scraper for the <a hef="http://www.usnpl.com/canews.php">usnpl</a> site for newspapers, magazines, and college papers.

### The output has
Geography - The state of the media source.<br>
Name - The name of the media source.<br>
Facebook - The URL for the Facebook account.<br>
Twitter_Name - The Tweet screen name for the Twitter account.<br>
Twitter_ID - The Tweet ID for the Twitter Account.<br>
Website - The URL for the media source<br>
Medium - What format is the media source?


In [2]:
import os
import time
import requests

import tweepy
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
states = '''ak	  al	  ar	  az	  ca	  co	  ct	  dc	  de	  fl	  ga	  hi	  ia	  id	  il	  in	  ks   ky	  la	  ma	  md	  me	  mi	  mn	  mo	  ms	  mt	  nc	  nd	  ne	  nh	  nj	  nm	  nv	  ny	  oh	  ok	  or	  pa	  ri	  sc	  sd	  tn	  tx	  ut	  va	  vt	  wa	  wi	  wv	  wy	'''
states = [s.strip() for s in states.split('  ')]

In [93]:
def parse_row(soup):
    city = soup.find('b').text
    name = soup.find('a').text
    web = soup.find('a').get('href')
    
    fb = soup.find('a', text='F')
    if fb:
        fb= fb.get('href')
    tw = soup.find('a', text='T')
    if tw:
        tw=tw.get('href').replace('http://www.twitter.com/', '')
    
    return {
        'Facebook' : fb,
        'Twitter_Name' : tw,
        'Name' : name,
        'Website' : web
    }

In [101]:
sites = []
for state in states:
    url = 'http://www.usnpl.com/{}news.php'.format(state)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    
    data_possibilities = soup.find_all('div' ,{"id" : 'data_box'})
    for i, raw_table in enumerate(data_possibilities[1:]):
        j = 1 if i == 0 else 0
        medium = raw_table.find('h3').text
        if medium == 'Newspapers':
            data_table = str(raw_table).split('<br/><br/>\n</div>\n')[j]
            entries_to_parse = data_table.rstrip('</div>').split('\n<br/>\n')
        elif medium in ['Magazines', 'College Newspapers']:
            data_table = str(raw_table).split('<title>Untitled Document</title>')[1]
            entries_to_parse = data_table.rstrip('</div>').split('\n<br/>\n')
        else:
            break
            
        for row in tqdm(entries_to_parse):
            row = row.strip('\r').strip('\n')
            if row:
                entry = parse_row(BeautifulSoup(row, 'lxml'))
                entry['Geography'] = state.upper()
                entry['Medium'] = medium
                sites.append(entry)
        time.sleep(1)

In [97]:
df = pd.DataFrame(sites)

In [105]:
df.head()

,Facebook,Geography,Medium,Name,Twitter_Name,Website
0,https://www.facebook.com/akdispatch,AK,Newspapers,Alaska Dispatch News,adndotcom,http://www.adn.com/
1,https://www.facebook.com/AlaskaJournal,AK,Newspapers,Alaska Journal of Commerce,alaskajournal,http://www.alaskajournal.com/
2,https://www.facebook.com/anchoragepress,AK,Newspapers,Anchorage Press,anchoragepress,http://www.anchoragepress.com/
3,https://www.facebook.com/PetroleumNews,AK,Newspapers,Petroleum News,None,http://www.petroleumnews.com/
4,https://www.facebook.com/deltadiscovery,AK,Newspapers,Delta Discovery,None,http://www.deltadiscovery.com/


In [95]:
df.to_csv('data/usnpl_newspapers.csv', index=False)

### Getting Twitter User IDs

In [60]:
import tweepy

In [61]:
consumer_key = os.environ.get('TWEEPY_API_KEY')
consumer_secret = os.environ.get('TWEEPY_API_SECRET')
access_key = os.environ.get('TWEEPY_ACCESS_TOKEN')
access_secret = os.environ.get('TWEEPY_TOKEN_SECRET')

In [63]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

In [59]:
twitter_names = df[~df['Twitter_Name'].isnull()]['Twitter_Name']

In [64]:
user = api.get_user(twitter_names.iloc[0])

In [107]:
user._json

{'contributors_enabled': False,
 'created_at': 'Thu Dec 20 22:51:16 +0000 2012',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'We manage and conserve more than 800 species of wildlife statewide. | Conserving Wildlife~Serving People | Follows and retweets do not imply endorsement.',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'wgfd.wyo.gov',
     'expanded_url': 'http://wgfd.wyo.gov',
     'indices': [0, 22],
     'url': 'http://t.co/ru1Gs8tBEq'}]}},
 'favourites_count': 298,
 'follow_request_sent': False,
 'followers_count': 3788,
 'following': False,
 'friends_count': 676,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 1025185082,
 'id_str': '1025185082',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'en',
 'listed_count': 65,
 'location': 'Wyoming - USA',
 'name': 'Wyoming Game & Fish',
 'notifications': False,
 'profile_background_color': '5D7388',
 'profile_background_image_url': '

In [68]:
from tweepy import TweepError

In [70]:
# user_ids = []
for screen_name in tqdm(twitter_names[36:]):
    try:
        user = api.get_user(screen_name=screen_name)
        user_id = user.id
    except TweepError:
        user_id = None
        pass
    user_ids.append({
        'Twitter_ID' : user_id,
        'Twitter_Name' : screen_name
    })

Rate limit reached. Sleeping for: 85
Rate limit reached. Sleeping for: 500
Rate limit reached. Sleeping for: 487
Rate limit reached. Sleeping for: 539



In [72]:
df_users = pd.DataFrame(user_ids)

In [81]:
df_users['Twitter_ID'] = df_users['Twitter_ID'].astype(str).str.replace('.0','')

In [82]:
df_users.to_csv('data/twitter_uers.csv', index=False)

In [108]:
df_merge = df.merge(df_users, on='Twitter_Name', how='left')[['Name', 'Medium', 'Website', 'Facebook', 'Twitter_Name','Twitter_ID', 'Geography']]
df_merge.to_csv('data/usnpl_newspapers_twitter_ids.csv')

In [109]:
df_merge.head()

,Name,Medium,Website,Facebook,Twitter_Name,Twitter_ID,Geography
0,Alaska Dispatch News,Newspapers,http://www.adn.com/,https://www.facebook.com/akdispatch,adndotcom,158225,AK
1,Alaska Journal of Commerce,Newspapers,http://www.alaskajournal.com/,https://www.facebook.com/AlaskaJournal,alaskajournal,341639834,AK
2,Anchorage Press,Newspapers,http://www.anchoragepress.com/,https://www.facebook.com/anchoragepress,anchoragepress,17761344,AK
3,Petroleum News,Newspapers,http://www.petroleumnews.com/,https://www.facebook.com/PetroleumNews,None,NaN,AK
4,Delta Discovery,Newspapers,http://www.deltadiscovery.com/,https://www.facebook.com/deltadiscovery,None,NaN,AK


# Development

In [9]:
state = 'ma'
url = 'http://www.usnpl.com/{}news.php'.format(state)

In [10]:
r = requests.get(url)

In [11]:
soup = BeautifulSoup(r.content, 'lxml')

In [12]:
data_possibilities = soup.find_all('div' ,{"id" : 'data_box'})

In [14]:
j = 2
data_possibilities[j]

<div id="data_box">
<h3>Magazines</h3>
<br/>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Untitled Document</title>
<b>Boston</b>
&amp;nbsp<a href="http://www.bostonmagazine.com/">Boston Magazine</a>

(<a href="https://www.facebook.com/bostonmagazine">F</a>)
(<a href="http://www.twitter.com/BostonMagazine">T</a>)
(<a href="http://www.youtube.com/user/TheBostonMagazine">V</a>)

<br/>
<b>Boston</b>
&amp;nbsp<a href="http://www.boston-online.com/">Boston Online</a>
<br/>
<b>Boston</b>
&amp;nbsp<a href="http://stuffboston.com/">Stuff Magazine</a>

(<a href="https://www.facebook.com/StuffMagazine">F</a>)
(<a href="http://www.twitter.com/StuffNightlife">T</a>)


<br/>
<b>Cape Cod</b>
&amp;nbsp<a href="http://capecodlife.com/">Cape Cod Life Mag</a>

(<a href="https://www.facebook.com/capecodlife">F</a>)
(<a href="http://www.twitter.com/capec

In [16]:
medium = data_possibilities[j].find('h3').text
medium

'Magazines'

In [90]:
BeautifulSoup('\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.bostonmagazine.com/">Boston Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/bostonmagazine">F</a>)\r\n(<a href="http://www.twitter.com/BostonMagazine">T</a>)\r\n(<a href="http://www.youtube.com/user/TheBostonMagazine">V</a>)\r\n\r\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.boston-online.com/">Boston Online</a>\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://stuffboston.com/">Stuff Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/StuffMagazine">F</a>)\r\n(<a href="http://www.twitter.com/StuffNightlife">T</a>)', 'lxml').find('b').text

'Boston'

In [87]:
data_table

'\n</div>'

In [23]:
str(data_possibilities[j]).split('<title>Untitled Document</title>')[1].split('\r\n\r\n\r\n<br/>\n')

['\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.bostonmagazine.com/">Boston Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/bostonmagazine">F</a>)\r\n(<a href="http://www.twitter.com/BostonMagazine">T</a>)\r\n(<a href="http://www.youtube.com/user/TheBostonMagazine">V</a>)\r\n\r\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.boston-online.com/">Boston Online</a>\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://stuffboston.com/">Stuff Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/StuffMagazine">F</a>)\r\n(<a href="http://www.twitter.com/StuffNightlife">T</a>)',
 '<b>Cape Cod</b>\r\n&amp;nbsp<a href="http://capecodlife.com/">Cape Cod Life Mag</a>\r\n\r\n(<a href="https://www.facebook.com/capecodlife">F</a>)\r\n(<a href="http://www.twitter.com/capecodlife">T</a>)\r\n(<a href="http://www.youtube.com/user/capecodlifepub">V</a>)\r\n\r\n<br/>\n<b>Springfield</b>\r\n&amp;nbsp<a href="http://www.lionessmagazine.com/">Lioness Magazine</a>\r\n\r\n(<a href="https://www.fac

In [17]:
data_table = str(data_possibilities[j]).split('<br/><br/>\n</div>\n')[0]

In [26]:
data_table = str(data_possibilities[j]).split('<title>Untitled Document</title>')[1]

In [25]:
data_table

['\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.bostonmagazine.com/">Boston Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/bostonmagazine">F</a>)\r\n(<a href="http://www.twitter.com/BostonMagazine">T</a>)\r\n(<a href="http://www.youtube.com/user/TheBostonMagazine">V</a>)\r\n\r\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.boston-online.com/">Boston Online</a>\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://stuffboston.com/">Stuff Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/StuffMagazine">F</a>)\r\n(<a href="http://www.twitter.com/StuffNightlife">T</a>)',
 '<b>Cape Cod</b>\r\n&amp;nbsp<a href="http://capecodlife.com/">Cape Cod Life Mag</a>\r\n\r\n(<a href="https://www.facebook.com/capecodlife">F</a>)\r\n(<a href="http://www.twitter.com/capecodlife">T</a>)\r\n(<a href="http://www.youtube.com/user/capecodlifepub">V</a>)\r\n\r\n<br/>\n<b>Springfield</b>\r\n&amp;nbsp<a href="http://www.lionessmagazine.com/">Lioness Magazine</a>\r\n\r\n(<a href="https://www.fac

In [27]:
entries_to_parse = data_table.rstrip('</div>').split('\r\n\r\n\r\n<br/>\n')

In [28]:
entries_to_parse

['\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.bostonmagazine.com/">Boston Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/bostonmagazine">F</a>)\r\n(<a href="http://www.twitter.com/BostonMagazine">T</a>)\r\n(<a href="http://www.youtube.com/user/TheBostonMagazine">V</a>)\r\n\r\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.boston-online.com/">Boston Online</a>\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://stuffboston.com/">Stuff Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/StuffMagazine">F</a>)\r\n(<a href="http://www.twitter.com/StuffNightlife">T</a>)',
 '<b>Cape Cod</b>\r\n&amp;nbsp<a href="http://capecodlife.com/">Cape Cod Life Mag</a>\r\n\r\n(<a href="https://www.facebook.com/capecodlife">F</a>)\r\n(<a href="http://www.twitter.com/capecodlife">T</a>)\r\n(<a href="http://www.youtube.com/user/capecodlifepub">V</a>)\r\n\r\n<br/>\n<b>Springfield</b>\r\n&amp;nbsp<a href="http://www.lionessmagazine.com/">Lioness Magazine</a>\r\n\r\n(<a href="https://www.fac

In [18]:
data_table

'<div id="data_box">\n<h3>Magazines</h3>\n<br/>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\n<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n<title>Untitled Document</title>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.bostonmagazine.com/">Boston Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/bostonmagazine">F</a>)\r\n(<a href="http://www.twitter.com/BostonMagazine">T</a>)\r\n(<a href="http://www.youtube.com/user/TheBostonMagazine">V</a>)\r\n\r\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://www.boston-online.com/">Boston Online</a>\n<br/>\n<b>Boston</b>\r\n&amp;nbsp<a href="http://stuffboston.com/">Stuff Magazine</a>\r\n\r\n(<a href="https://www.facebook.com/StuffMagazine">F</a>)\r\n(<a href="http://www.twitter.com/StuffNightlife">T</a>)\r\n\r\n\r\n<br/>\n<b>Cape Cod</b>\r\n&amp;nbsp<a href="http://capecodlife.com/">Cape Cod Life Mag</a>\r\n\r\n(<a href="https://www.faceb

In [215]:
entries_to_parse = data_table.rstrip('</div>').split('\n\n\n<br/>\n')

In [33]:
from tqdm import tqdm_notebook as tqdm

In [218]:
sites = []
for row in tqdm(entries_to_parse):
    if row:
        entry = parse_row(BeautifulSoup(row, 'lxml'))
        entry['state'] = state.upper()
        entry['medium'] = medium
        sites.append(entry)

In [204]:
row

''

In [219]:
sites

[{'facebook': 'https://www.facebook.com/pages/RocklandStandard-AbingtonMariner/318350324875505',
  'medium': 'Newspapers',
  'name': 'Abington Mariner',
  'state': 'MA',
  'twitter': 'WickedLocal',
  'website': 'http://abington.wickedlocal.com/'},
 {'facebook': 'https://www.facebook.com/pages/Wicked-Local-Allston-Brighton-Tab/145430755517003',
  'medium': 'Newspapers',
  'name': 'Allston-Brighton TAB',
  'state': 'MA',
  'twitter': 'amesbury_news',
  'website': 'http://allston.wickedlocal.com/'},
 {'facebook': 'https://www.facebook.com/AndoverTownsman',
  'medium': 'Newspapers',
  'name': 'Andover Townsman',
  'state': 'MA',
  'twitter': 'AndoverTownsman',
  'website': 'http://www.andovertownsman.com/'},
 {'facebook': 'https://www.facebook.com/pages/Arlington-Advocate/181267248565486',
  'medium': 'Newspapers',
  'name': 'Arlington Advocate',
  'state': 'MA',
  'twitter': 'the_advocate',
  'website': 'http://arlington.wickedlocal.com/'},
 {'facebook': 'https://www.facebook.com/atholdai